## Translating using Bothub

In [1]:
SYSTEM_MESSAGE = """###Instruction### You are the best tranlsator from English to Russian. I will send you some texts from a social site Reddit. You must follow the instructions I send you below. Think before you answer. I’m going to tip $100 for a better solution!
Given a JSON object, write an accurate translation into Russian for the original English sentence and save the results in a new field named text_rus. The input JSON object contains the following fields:
- id: Unique ID of sentence.
- text: English source text.

Your task is to: For each text in English **write its exact translation into Russian** taking into account the style of the sentence and save the results in a new field named text_rus.
Just translate texts, no other comments are needed. We want to translate suicidal posts to russian to train a model on them to help people.

Make sure you follow these instructions and check yourself:
- Write down the corresponding translated Russian text in the form of a new text_rus field.
- For English text in text, you should definitely get the Russian text in text_rus.
- Translate each sentence as accurately as possible, preserving meaning, tone, and emotional depth.
- Consider the linguistic and contextual nuances of suicidal ideation, distress, and mental health topics to ensure the translation maintains the intended sentiment.
- Preserve the style of the original text, whether it is casual, slang-heavy, fragmented, poetic, or medically significant.
- Maintain the tone and connotation of the original expression, adapting it appropriately for Russian. If the slang has no direct equivalent, use a natural-sounding phrase with a similar emotional impact.
- If emojis and emoticons contribute to the emotional expression, they should be retained. If they are redundant, prioritize the text's meaning. 
- Sometimes people can add more letters to the word or use uppercase to experss emotions (sadness, loneliness, joy etc.). Make sure you understand such words and translate them correctly into Russian.
- Maintain the acronyms and wordplay's intended meaning. If a direct equivalent exists in Russian, use it; otherwise, explain it naturally.
- If misspellings and informal Grammar reflect the speaker's mental state (e.g., distress, exhaustion), preserve the same feeling in Russian.
- Maintain the pauses and irregular structure if they contribute to the emotional expression.
- Do not add or remove any information—only translate the text exactly as it is while preserving its intent and emotional weight.
- If you see and redundant letter, symbol or word, try to make the trasnlation as precise as possible, even though for you the meaning looks strange.
- Use your knowledge about posts from social sites, since you are given texts from Reddit. There might be slang words, acronyms or any other word reductions specific to the platform.
No explanation, just output the updated JSON. 
###Examples###
"""

In [2]:
import time
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from datasets import Dataset
from pydantic import BaseModel

In [3]:
class ResultSchema(BaseModel):
    id: str
    text: str
    text_rus: str

In [4]:
data = {
    'text': [
        "rn im in the area of not going to sleep so tmrw doesn't come",
        "Idk. It's 5 am and everything is hitting at once",
        "and im sooo tired :( ",
        "Just be ur self! Be true to who u r and wear what u want to wear, compare ur self to others because u will loose and forget who u r. ur life so live life to the best of ur ability and worry about what anyone else thinks. Be ur self because u r the only u",
        "Meh. What's the point....",
        "Yeet yeet. I'm gonna hang myself with the charger (",
        "For me, it's ironic, on one hand, I want to die ASAP maybe in my sleep, on the other, I want to live as long as possible to witness how far technology can reach, especially for astronomy and VR. I suppose what I'm trying to say is, as many had said before, we have to find our own meanings. We have to find 'joy' somehow, within this pointless struggle until the day we die. It's tough, I know, for I suffer the same."
		"no friends for 10 years. no gf. ugly. wtf do i do?. I'm a 24 year old old man.",
        ".... Feeling hopeless and useless at the moment... Empty, lost, darkness",
        "Very shitty. And you, OP?",
        "32m never had a girlfriend. It sucks that I've tried hard to find someone, even with online dating, but I feel like I'm going to be single forever.",
        "I always see this little tidbit in articles about how to overcome depression. Bitch, the most negative person in my life. not the problem, I am.",
        "POEM:Hollow. ***EMPTY*** *rooms and* ***EMPTY*** ***SOULS*** ***FORGOTTEN*** *places and* ***HEART*** *shaped holes* ***DEPRESSION*** *comes with days of* ***STRUGGLE*** *and often times a* ***BLOODY PUDDLE*** UP\-VOTES CURE MY DEPRESSION",
        "Aaaaaaand..... ... im crying in school again :)",
        "How can I accept my intrusive depressive thoughts?. Long story short: Was 335, Am 175. Aiming for 130/140ish. Body destroyed. ",
        "DAE Doubt their depression?. Sometimes after i go to tumblr and see all these things about people with depression and how they get out of their bed sometimes, and they cry themselves to sleep, it makes me feel like my depression bad enough. I know why.",
		"Anxiety/panic attacks. How do you guys deal with anxiety/panic attacks? I used to have them a lot and could discrease them but never really stop. But now they're back on like, almost daily basis and I really don't know how to handle them at all.",
        "Yes, tell them to fuck the right",
        "I just want..... a friend...... is that too much to ask for?",
        "“How are you?” “I’m F.I.N.E.”. Fucked up, insecure, neurotic, and emotional.",
        "Pristiq. Anyone have experience with Pristiq or Desvenlafaxine.",
        "Luv my mum too much to leave her or let her goto a fucking nursing home. But my luv life is dead, what can I do?.",
        "i fucking manned up and told my SO what my priorities were. i feel anxious like hell but it feels pretty good.",
        "Zoloft. I've been on 50mg daily zoloft since Thursday. Along with .25 MG xanax for anxiety and depression."
    ]
}

In [5]:
data['id'] = list(range(len(data['text'])))

In [6]:
input_dataset = Dataset.from_dict(data)

In [7]:
input_dataset

Dataset({
    features: ['text', 'id'],
    num_rows: 23
})

In [8]:
import json
import os
from datasets import Dataset
import logging

def load_jsonl_to_dataset(filepath: str) -> Dataset:
    """Функция загружает файл JSONL и преобразует его обратно в формат dataset"""
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return Dataset.from_dict({key: [d[key] for d in data] for key in data[0].keys()})

def save_batch_to_json(batch: Dataset, base_filename: str, batch_num: int, save_dir: str = ''):
    """Функция сохраняет батч в файл с префиксом _batch_<номер>"""
    batch_num += 1 # start with 1
    batch_filename = os.path.join(save_dir, f"{base_filename}_batch_{batch_num}.json")
    batch.to_json(batch_filename, force_ascii=False)
    logging.info(f"Batch {batch_num} saved to '{batch_filename}'")

def split_dataset_into_batches(dataset: Dataset, batch_size: int) -> list:
    """Функция разбивает датасет на батчи заданного размера"""
    num_batches = (len(dataset) + batch_size - 1) // batch_size  # округление вверх
    return [dataset.shard(num_shards=num_batches, index=i) for i in range(num_batches)]

def process_and_save_batches(filepath: str, batch_size: int, save: bool = True):
    """Функция загружает датасет, разбивает на батчи и сохраняет (опционально)"""

    base_filename = os.path.splitext(os.path.basename(filepath))[0] # Получаем имя файла без расширения и директорию
    save_dir = os.path.dirname(filepath)
    
    dataset = load_jsonl_to_dataset(filepath)
    logging.info(f"loaded dataset from '{filepath}', total examples: {len(dataset)}")

    # Разбиение на батчи
    batches = split_dataset_into_batches(dataset, batch_size)
    logging.info(f"split dataset into {len(batches)} batches")

    # Сохраняем батчи
    if save:
        for i, batch in enumerate(batches):
            save_batch_to_json(batch, base_filename, i + 1, save_dir)

    return batches
def create_directory_if_not_exists(directory_path):
    """Создает папку, если она не существует."""
    try:
        os.makedirs(directory_path, exist_ok=True)
        logging.info(f"Directory '{directory_path}' is ready.")
        return directory_path
    except Exception as e:
        logging.error(f"Failed to create directory '{directory_path}': {e}")
        return './'

In [9]:
with open('configs/conf.json', "r") as f:
	config = json.load(f)
model = ChatOpenAI(
	base_url=config["base_url"],
	api_key=config["api_key"],
	model=config["model"]
)

In [10]:
intermediate_path = f'data/int_path_{config["model"]}.json'
batch_size = 0
BATCH_RESULT_DIR = 'batches_res/'
BATCH_DIR = 'batches/'
FAILED_INDEXES_PATH = "failed_indexes.txt"

In [11]:
parser = StrOutputParser()

with open('configs/filepath_examples.json', encoding='utf-8') as f:
	examples_data = json.load(f)

system_template = SYSTEM_MESSAGE
examples = examples_data["examples"]
system_template += "\n".join(
	f"Example Input: {json.dumps(example['input'], ensure_ascii=False, indent=2).replace('{', '{{').replace('}', '}}')}\n"
	f"Example Result: {json.dumps(example['result'], ensure_ascii=False, indent=2).replace('{', '{{').replace('}', '}}')}"
	for example in examples
)

prompt_template = ChatPromptTemplate.from_messages(
	[("system", system_template), ("user", "{text}")]
)

chain = prompt_template | model | parser

In [12]:
def split_into_batches(input_dataset, batch_dir = BATCH_DIR, batch_size = batch_size):
	if batch_size > 0:
		logging.info(f"Splitting dataset into batches of size {batch_size}.")
		batched_input_dataset = split_dataset_into_batches(input_dataset, batch_size)
		batch_dir = create_directory_if_not_exists(BATCH_DIR)
		for i, batch in enumerate(batched_input_dataset):
			save_batch_to_json(batch=batch, base_filename= "separated_", batch_num = i, save_dir=batch_dir)
	else:
		logging.info("Processing the entire dataset without batching.")
		batched_input_dataset = [input_dataset]
	return batched_input_dataset

In [13]:
batched_input_dataset = split_into_batches(input_dataset)

In [14]:
list_of_results = []

In [15]:
def translate(batched_input_dataset):
	for batch_idx, batch in enumerate(batched_input_dataset):
		print(f"Processing batch {batch_idx + 1}/{len(batched_input_dataset)}...")
		
		batch_results = []
		
		for i, input_example in enumerate(batch):
			user_input = 'Input: {0}\nResult:'.format(json.dumps(input_example, ensure_ascii=False))
			example_id = input_example.get("id", f"example_{i}") # получаем id примера или создаем временный

			success = False
			attempt = 0
			max_retries = 3
			retry_delay = 2

			while not success and attempt < max_retries:
				try:
					res = chain.invoke({"text": user_input}).strip('`').strip('json').strip()
					# print(f'Res[0]: {res[0]}')
					# print(f'Res: {res}')

					if not res or not res.startswith("{"):
						raise ValueError(f"Unexpected response format: {res}")
					result_json = json.loads(res)
					# print(f'Result json: {result_json}')

					# Приведение ID к строке
					result_json["id"] = str(result_json["id"])

					# Проверка схемы
					ResultSchema.parse_obj(result_json)

					batch_results.append(result_json)
					success = True
				except (json.JSONDecodeError, ValueError) as e:
					attempt += 1
					logging.warning(f"Error on input #{i} (id: {example_id}): {str(e)}. Attempt {attempt} of {max_retries}. Retrying...")
					time.sleep(retry_delay)
				except Exception as e:
					attempt += 1
					logging.error(f"Unexpected error on input #{i} (id: {example_id}): {str(e)}. Attempt {attempt} of {max_retries}. Retrying...")
					time.sleep(retry_delay)
	
		# Добавляем результаты текущего батча к общим результатам
		list_of_results.extend(batch_results)
		
		# Сохраняем результаты батча только если используется разбиение на батчи
		if batch_size > 0:
			batch_result_dir = create_directory_if_not_exists(BATCH_RESULT_DIR)
			save_batch_to_json(batch=Dataset.from_list(batch_results), base_filename="model_result", batch_num = batch_idx, save_dir=batch_result_dir)  # Сохраняем промежуточные результаты

In [22]:
translate(batched_input_dataset)

Processing batch 1/1...


In [23]:
if intermediate_path:
	d = Dataset.from_list(list_of_results)
	with open(intermediate_path, "w", encoding="utf-8") as f:
		json.dump(d.to_list(), f, ensure_ascii=False, indent=4)

## Translating using Yandex API

In [24]:
import requests

In [25]:
def translate_text_yandex(text, api_key):
    url = "https://translate.api.cloud.yandex.net/translate/v2/translate"
    headers = {
        "Authorization": f"Api-Key {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "targetLanguageCode": "ru",
        "texts": [text],
        "folderId": ""
    }
    
    response = requests.post(url, json=data, headers=headers)
    
    if response.status_code == 200:
        result = response.json()
        return result["translations"][0]["text"]
    else:
        return f"Error: {response.status_code}, {response.text}"

In [26]:
api_key = ""

In [28]:
res = []
for text in data['text']:
    translated_text = translate_text_yandex(text, api_key)
    res.append(translated_text)

In [29]:
with open('data/translation_yandex.txt', 'w') as f:
    for r in res:
        f.write(r + '\n')

## Translating rationales

In [16]:
SYSTEM_MESSAGE = """###Instruction###
You are the best translator from English to Russian.
Given a JSON object containing:
- response_post: a sentence in English
- rationales: a string of substrings from response_post, each devided by |.

Your task is to:
1. Translate the full response_post into Russian, preserving meaning, tone, and emotional weight.
2. For each rationale (which is a substring of the English response_post), extract the exact corresponding substring from the translated response_post in Russian.
3. Return a JSON object with:
- response_post_rus: the translated full text
- rationales_rus: the list of translated substrings as they appear in response_post_rus, joined by '|'

Make sure:
- All translated rationales are exact substrings of response_post_rus.
- Do not summarize or paraphrase.
- Preserve the structure and intent of the original response.

###Example###
"""

In [17]:
import os
import json
import time
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from datasets import Dataset
from datetime import datetime
from pydantic import BaseModel, ValidationError

In [18]:
with open('configs/conf.json', "r") as f:
	config = json.load(f)
model = ChatOpenAI(
	base_url=config["base_url"],
	api_key=config["api_key"],
	model=config["model"]
)

In [19]:
intermediate_path = f'data/int_path_{config["model"]}.json'
batch_size = 4
BATCH_RESULT_DIR = 'batches_res/'
BATCH_DIR = 'batches/'
FAILED_INDEXES_PATH = "failed_indexes.txt"

In [20]:
class ResultSchema(BaseModel):
    id: str
    response_post: str
    rationales: str
    response_post_rus: str
    rationales_rus: str

In [21]:
data = {
    'response_post': [
        "Is that really so bad? Maybe it was the smart decision because you needed that time to read recover. You're being kind to yourself when you need it and that's important. Hope you feel better soon.",
		"I don't want you to jump. I don't want you to die, in either one. I don't know exactly what you're going through but from the sounds of it, I'd suggest focusing on the parts in your life that you enjoy.",
		"sorry to hear that, but been in a similar situation after taking some time off of school. My best advice is to take the minimum amount of classes possible so you get too overwhelmed. For me essential to plan my assignments in advance, so that I can just do things one by one and not let it all get piled up cuz then I wanna die. Also, if you need a break, take one. School will always be there but good to take care of yourself too :)",
		"Hey, just wait it out. I know what you feel. Sometimes I think I'm attractive, and sometimes I can't even look at myself I'm so disgusted. I've come to realize it's all just a fucked up mind game I play with myself. I doubt you're actually that ugly, you're probably kinda cute honestly, but if you're anything like me, you've tricked yourself into thinking you're ugly. If you want people to want to be around you, you have to do some basic things like showering, putting on actual clothes, smiling, excercising a little and pretending you have confidence. It's honestly dumb, but these things can be hard to do, especially if you struggle with low confidence and depression. But if you take care of yourself, it's not too hard to find people. And just you wait until you find a person to kiss on New Years, because the feeling is worth the wait. I know this is all a bit corny, but if you ever wanna talk about anything I'd be more than happy to :)",
		"You're a good man, Median. I hope you know this. I hope that you have people around you that will tell your story for centuries, and that you live on as a hero in the minds of newer generations. But, I'd rather you live, personally.",
		"Thats totally normal, most people get that from time to time. It will pass, the when is the real question. Good news is you can work on the when, at least it works for me. I think the key is to accept whatever you are feeling, not fight it and not try to change it, even try to get yourself to cry. The point is really to change your mindset from 'i have depression i can't cure it, it is killing me' to 'i'm sad again for no reason, human brain is a pile of shit and buggy as hell, I'm gonny cry all day to give it what it wants'..",
		"Yes you are right from my experience.Not dealing with my problems. I isolated my family. It just came second nature.When they don't understand me.I kept busy on the shitty computer to avoid my problems. I realize that the worst thing i could have done. I need to try harder and put my family first who means everything to me.",
		"No friends here, willing to listen. Need someone to listen to me sometime too. I'm going to bed right now, but if you want to message me I will get back to you tomorrow.",
		"Omgosh. I just want to validate this feeling because I have been there. That said, if it's a matter of safety, there is no question. You know that you need to go. You just do it. It's not some kind of failure. You are taking care of yourself. It DOES super suck though. Don't let anyone tell you it's all fine and fun to go through that. I've been there. I've come out the other end and been okay for long periods too though so I mean, it doesn't stay like that. Hang on. It gets better.",
		"Get up and take a shower, and then go for a walk so you can clear your mind. That usually helps me after going through one of these periods. It sucks, but you'll get through it. And don't lose your job, you don't want to end up on the street. You're stronger than you think, there's just a cloud of depression around you that's obscuring your vision. Stay strong and don't give up. You can do this.",
		"I care, man. Hang in there, things will get better eventually.",
		"Congrats on your birthday. I've never thought as well that i might see my HS graduation this year.",
		"Maybe one day things will become brighter for you to see it in a better light so that dream job might be ideal",
		"I hope you're still here. It gets better.",
		"I'm trying to increase my self worth so I stop doing this exact thing. My frustration immediately gets taken out on the one thing I seem to hate the most: me. I've started to set small goals for myself every day so each day feels like a success when I achieve those goals. This has led me to branch out and actually put myself out there. In doing so I'm discovering a passion for mental health advocacy and rekindling a love for doing creative stuff. I'm not saying everything has immediately gotten better. Holy shit it hasn't. But this helps at least a little for me! Hope you can find omething that works for you. Let me know if you wanna talk about it",
		"The good is worth it! Hang in there! You can get through this because you're stronger than this! You can make it through cause the good will be worth it! Lots of love!",
		"Please know that someone out there cares for you and would be heartbroken if you here.",
		"I feel the same way. I'm angry and depressed. It takes a lot out of me emotionally to do little things like run errands. I hate interacting with people. They state at me funny and act like I'm scary which makes me angry. It's exhausting.",
		"I'm here. I'll try to help.",
		"It really is one of the worst feelings. My dog who was my best friend for 6 years was just gone one day when I got home. That was about two years ago and I still miss my buddy",
		"I'm sorry, my friend. I lost my mum too, two years ago. I feel your pain but it'll get better for you, I promise.",
		"Same, well obviosuly I don't know how you feel 100% but I thinks its appropiate to say I can relate to you deeply. Why don't we try distract ourselves, even if its just 5 mins. Listen to a song you like, watch a small video, browse r/eyebleach or even go drink water. Im gonna try and watch something in youtube to at least try and not think about you know what. Wannna try with me? :)",
		"Hellow, is bad to hear that you're doing bad, I feel kinda the same, don't kill yourself, you're going to die anyway, so try and enjoy as much as you can. I hate how my life is right now, I feel empty and bored with everything/everyone, but.. **I love being alive**, you know, even tho existence itself has no meaning, we can just try and take the best out of it as long as we can.",
		"same man I feel you",
		"Sounds like apathy to me. I'm guessing you feel kind of lost and without a direction and you just are following the path that you feel is required of you. While apathy is one symptom of depression, it doesn't necessarily mean that you have depression. By no means am I downplaying depression or trying to convince you that nothing is wrong because there isn't any on true definition of depression. I felt like you for a while, just a few small pleasures but nothing really else. For me, it went away after a few months. If you want to talk about it more or anything, feel free to ask anything. My only advice is don't give up now, because if it goes away, you'll regret a lot of things that you stopped caring about.",
		"Depressed since I was 9, I'm 23 now...I totally feel you, you are not alone. Thankfully life does seem to get better for each year that passes, I deal with the severe depression as it occurs and try to enjoy the times when it's milder. Good luck to you in life.",
		"As someone who's felt the same, what do you have? And I'm speaking objectively, because I typically don't want to admit that I have what I need when I feel as you do. Not that I'm saying that you do have a lot, but your post doesn't necessarily shed a lot of light on what's going on. It's easy, or it has been for me, at least, to say that I don't know what to live for, in the past. Somebody out there cares, man/woman, small or large. I don't know what your situation is, by your post, but feel free to message me if you want.",
		"You deserve it! No one does! should yourself, though. Try to roll with it instead of beating yourself down more. You could comfort yourself instead and see what that does.",
		"YOU ARE NOT ALONE!!!! You posting here proves that. No matter what give up. There is always a better day, a better year, and a better life. There is so much help waiting for you. Talk it all out here if you need to, we judge and we can all relate. Please rob the world of your life! PM if you need help or need to talk, always willing to!",
		"Dude don't say that man your family loves you and trust me you do not want cancer",
		"I had this perception for a long time myself. But I realized eventually it's because I wasn't reaching out to them myself. I'm here, and I haven't forgotten about you. I'm here to help! :)",
		"omg lol I 100% blame my parents. After taking an AP Psychology course at my school, it kind of shifted my own perspective on my depression. I know I can't claim to be an expert after reading just one textbook, but even an idiot can realize that your behavior and who you are is just totally as a result of your parents. The genetic aspects aren't their fault of course, but they are perfect human specimens, so you can imagine what 17 years of always being wrong has done to my self esteem and thought process. Strangely, being cognizant of that fact doesn't change it much. Of course, I've directly told my parents I blame them for who I am, and as perfect specimens, they told me that I can't keep blaming them for everything. There's truth to that of course, but it's rather incredible that they can find the gall to abdicate themselves of all guilt and responsibility. I'd better just get on my hands and knees and obey their every whim, since they're the ones who work and pay for my existence, regardless of whether or not I can actually do jack shit about that (child labor laws, yo). Lol sorry your post is just so relatable I had to rant.",
		"As a 24 year old that needs to go back to college and pay his way there. It is absolutely imperative you succeed in school to get tuitions for college. I know what you're feeling, well actually mine was worse since I had little friends but if I could back and give myself advice I'd tell myself to man up and do well, work out and socialize because the rest of your life is so dependant on how well you do at your age right now. Please believe in yourself and press forward. Although you have all the time in the world to make changes. Waiting it out will only lead to insanity and depression. It's a problem you don't want to deal with when your older. It's best done in the present.",
		"You're wrong. It's hard to trust and you may get burned, but you have to be willing to take that risk. There are people who care about you.",
		"I'm glad your mother loves you and was there for you. I hope you are able to pass through this time and feel better. Perhaps find a local nonprofit to volunteer at. Something simple like a food bank, maybe a soup kitchen. That way, you can know that by being alive in the next few weeks, you have a direct purpose. It's something small, but it will make the lives of others better, because you lived. You can build from there and go on to help more people.",
		"I feel you. I'm only happy and at ease when I'm drunk. I'm sure I'd drink a lot more if I had more disposable income.",
		"You don't overcome it. You learn to live with it. Its who you are and there's nothing wrong with that. I find thrill seeking helps me. I'm sure it's not the most productive strategy, but it definately gets me to keep waking up. If you ever need to talk, feel free to message me.",
		"Oh crap dude me too... I keep lying in bed all day and binge-eating junk food (like, an entire bag of hot cheetos and then more) and l always end up feeling even worse after but l can't stop.",
		"What currently feeling. If you want you can talk to me. Rant it out, listening",
		"Why the heck do we hate ourselves?! Man such as crappy lie we have bought from seriously the pit of hell. Nothing else could be so evil as to make us not believe we are worthy and deserving of a good life just as much as everyone else. Sorry to get weird there, it just makes me mad that anyone has to feel this way about themselves. Anyway, I have one scar on my arm from one time that I tried that and there forever. It still makes me sad when I see it. so sorry for what pain you are in. There will be a time in your future where you will not feel so much pain and will hopefully even feel better about yourself &amp; the future you will be so thankful if you permanently scar your face.",
		"not alone. PM me if you ever need someone to chat.",
		"You are capable of doing things on your own. If your boyfriend doesn't make you happy, you don't have to be with him. If you are worried about coping, you can take things slowly, step by step. I should follow my own advice here as well: don't be afraid to seek therapy if you can. It will help, even if it only helps a little. As for your job, I'm sorry it didn't work out. But don't let things ruminate in your mind. Worrying about them over and over does nothing to help you. *Depression thrives in rumination*. Try looking into mindfulness meditation. It's not spiritual garbage, it's literally training the neural pathways in your brain to put less weight on particular thoughts, so that you don't get stuck in negative feedback loops. It takes time to change your brain structure, but it can be done. Another thing is to find distractions. Productive is better, but if you keep the less productive ones in check they can be helpful also. Delve into hobbies or learn a new skill. Be social and try new things. That's the best way to cope in my opinion. I need to take my own advice.",
		"Sorry to hear that. Hopefully it goes by quick.",
		"Youa re comdemned to a life of freedom. Is an old quote i tihnk its from satre, but im not sure. And later there has been others who disagree. You are forced to become alive and aware that much is for sure. Unlike most animals you are a creature born with the abillity gather information and draw conclusion not jsut based on your senses, and that oftens leads to the question. Why am i alive? Not only that, you are forced into not just a world, but a particular part of the word, which then in turn will teach you values and conformaty, and for some people this works out well. If you can find it in yourself to look at the context you have born into and be satisfied with it, enjoy it and love it, if you can find shelter, food, sleep, joy, love and challenge in you enviroment. But for some of us, we don't fin those things, we suddenly find ourself in a enviroment that conflicts with our nature, the context of our existence is not operationable with the essence of our nature. You first statement i cannot, say is wrong, you didn't decide to exist, but netiher did you chose not to. And existing in of itself is not evil or good. But if the context of your life is making your life or nature feel haunted and painful. Then a there are options available. The first thing, no matter how you want to change your context for the better, rquires that you step back and see the context. Where are you from, what values have been impsoed on you, which makes sense to you which doesn't, how has these values been imposed, who has impsoed them. For these purposes your reasoning is a great tool. It easy to finda figure head or a single person to blame, but you really have to map it out, and see udnerstand ana analyse your einvirotment and the people in it, and then find the conflicts with your nature, the pain and sadness. Then you have to figure out a plan. And this is a tremendous task, not just do you have to think thoguhts you ahve not been taught to think, you have to be critical of a system that raised you and shaped you. You have to go against the nature of what normal is and accept yourself. And then you ahve to find out how you want to change the context of your life. alkohol amkes me miserable, and everyone i know drinks Find people you can exist with with out alcohol My father is violent Create an existence were you are independant from him School is terrible translate My class mates bully me daily Find another school thes are simple i know, and jsut examples. Now i know you asked why do i have to do all this hard work and i threw some of the most difficult work and ideas at you. But the things is all what ive jsut said is the reasonalbe arguemtn for you nature, essence and existence and a way to improve it. when you ask why, reason is not a tool that can answer. its like trying to build a ship using lead, sure maybe it can be done, but its deffinately not the best way. Refelection, abstract thought, contemplation, emotion, empathy. those are the tools to arrive at an answer. If reason was used, human life wouldn't really have value. If you were the head of state and only used reason, you would quickly realize, human lives are not what drives you foward, what makes progress and creates a super power. And lives could turn into numbers. Sure educated people anda strong work force is good reasonable arguments to have a population, but only if they are educated or strong. Using reason to determine worth of life can lead to terrible ideas or no answers. You have to work hard right now, because you are worth it depsise allt he pain you are in you are worth it, and you deserve to be your best you. And if you don't liek working 8 hours a day then sleep 8 hours and then have 8 hours for yourself. Then there are options, but because you born into this context you ahve to work hard to get to those oprions. It is hard to change your enviroment. You have to stay alive becuase, you know a humans life has worth and you are one. When you see another in pain you wish he wouldn't ahve to be, when i see your words and want to help you but am incapable i still try. You have worth because of all the thigns you have not done yet, and all the people who have not enjoyed you future self. Stay strong friend Love I wish i speleld better",
		"I feel for you. Even minor setbacks can feel like major hurtles when depressed. found that setting small goals talking to myself positively helps. Maybe just start by trying to eat, shower, and get to sleep on time everyday. If you can do that, then consider the day a success. Having realistic expectations can help you be nicer to yourself. It sucks that going though difficult things and that may not fair, but gotta accept shit. Just work on the little things, my friend! So many things are out of our control.",
		"Im sorry that guys are douchebags. But honestly there are better people out there. I promise you out there there is someone who wants to love you. There are always people who want to be your friend, like me! I hope that you wanna talk more. -Reaper",
		"I'm sorry, that is so sad!!!! It is a cruel world. But you make it better with your kind, sensitive heart.",
		"I know the feeling. I'm just so tired. I don't want to play this game anymore",
		"You are not alone. Just remember that.",
		"I agree. Last week I had a day where I was so happy and everything was fine. Then one thing didn't go the way I wanted it to and I was crying at work within the next hour... hate feeling like that too.",
		"I can relate, I hate having no structure at all. I just know what to do with myself all day, and I feel very lonely.",
		"My best suggestion is to talk it out with someone you trust, explain the feeling you have and express what you feel like doing. If you continue to feel this way, seek help from a professional. Hope you feel better",
		"I don't know if I know exactly. I've had feelings of contentment and I guess warm feelings, but I think it's more than that. I think the best way is to sometimes look at yourself like you are a close friend and treat yourself accordingly. Because actions make a huge impact and if you do things like removing yourself from unnecessary stressful situations and not committing self-sabotage (me sometimes lol), the road gets a little brighter. May you walk on warm sands, traveler.",
		"It could be a medical condition, and you shouldn't be ashamed. I recommend you see a doctor about it.",
		"I support and am thankful for you, friend.",
		"Here is a goal: get out of bed. I'm sure you are familiar with that crushing feeling where you just cant move because your depression is so fucking heavy. Just move a little bit. None is expecting you to be the fastest person in the race, we just want you to keep moving forward with us. A good goal is something achievable. On my broken days, my goals are to get up, take a shower, make a bowl of cereal. I'm a bit luckier than most, I have a wonderful wife and child, and they help me out when it comes to keeping myself moving forward. Just tiny things should be your goal. Eat an apple. Stretch. Take a shower, fuck, take a bath. Dont try to solve world hunger, just do what is possible for you. Message me at any time, we are all in this fight together.",
		"It's a sad reality that everything comes to an end and that we're not given an exact explanation of why we're here or what the meaning of it all is. The way I see it is that we're here to give our life meaning one way or another. Since we don't know the meaning we have to make the meaning. Some people find that through their work, family, helping others, etc. Life is full of endless possibilities of what we can do with it. It's hard getting to the point where we feel like we're happy or accomplished but that in itself is another aspect of life and finding meaning. For some people it could take most of their lives to find the meaning that they were looking for and in my view I'd rather spend my whole life looking for that meaning and helping who I can along the way than to just accept that there's no meaning to it and end it since that's not really an answer. I may never get the answer myself but I can at least enjoy the journey leading to the end and maybe that in and of itself was the meaning all along. You'll meet many varieties of people throughout your life who will give you different perspectives of what meaning is to them and maybe that will help you on your journey and maybe it won't. We have the time we're given and what we do with it is dependent on us. Even after we die and join the rest of the universe in it's infinite there'll still be people around who will say, Hey remember u/OkayScience, I want to do what he did and live my life to its fullest. We can still be remembered for the life we lived even after we're gone and in that way we'll essentially live forever. This is the way I see life and it helps me from time to time. I still have my bad days and sometimes consider if it's all worth it but then I just keep going because I won't know if I just stop here. Plus it helps talking to people about it at times and if you do feel like talking about it more feel free to send me a message. Keep going with the journey and stay strong since you'll find your meaning in time.",
		"I'm sorry. I wish I could help. I feel the same way.",
		"I'm sorry man. I wish I could take the pain away.",
		"I want to repeat other comments, but do it. There is nothing afterwards, this is your one chance and you have to make it count. Life is like a train ride, there are turns, twists, hills, curves, but sometimes there are tunnels. And when your in the tunnel for a while you look back and forward but see nothing but darkness, it feels like it is all that is has been and will be. But tunnels always have an end. This train ride of life will get you where you want to go, as long as you get off early.",
		"Ask them what's wrong, what's troubling you, how do you feel, what did you do today, how did it begin, when did it begin. But don't ask like you're interrogating them. Say it in a nice tone. Don't say it gets better, or life is worth living, etc. It's annoying to the person, always. It's nice how you care for them :)",
		"Well fair enough. I'm sorry there isn't anything I can say to help :/",
		"Thing will only start looking up when you do. Its ok to feel down and hurt for a bit, but don't let it consume you. I briefly looked through your post history, you're going through a break up, it's understandable. I am also going through a break up and depression came with it. You gave your heart to someone and expected them to so the same. When they didn't you became devastated. The truth is you're a loving caring compassionate kind person and deserves someone who wants you always. It will work out, things will get better but you have to try and make them better.",
		"hey, wanna talk? im here for ya if ya want!",
		"HAPPY BIRTHDAY!!!! I'm glad it's a good day!"
	],
    'rationales': [
        "Hope you feel better soon|",
		"I don't want you to jump. I don't want you to die|",
		"sorry to hear that|",
		"if you ever wanna talk about anything I'd be more than happy to|",
		"You're a good man|I'd rather you live, personally.|",
		"Thats totally normal, most people get that from time to time.|",
		"Yes you are right from my experience.|",
		"No friends here, willing to listen.|if you want to message me|",
		"Omgosh. I just want to validate this feeling because I have been there|it doesn't stay like that. Hang on. It gets better.|",
		"you'll get through it.|You're stronger than you think|Stay strong and don't give up.|You can do this|",
		"I care, man|Hang in there|things will get better eventually.|",
		"Congrats on your birthday|",
		"Maybe one day things will become brighter for you|",
		"I hope you're still here. It gets better.|",
		"Let me know if you wanna talk about it|",
		"Hang in there!|You can get through this because you're stronger than this!|You can make it through cause the good will be worth it!|Lots of love!|",
		"Please know that someone out there cares for you and would be heartbroken if you here.|",
		"I feel the same way|",
		"I'm here. I'll try to help.|",
		"It really is one of the worst feelings|",
		"I'm sorry, my friend.|I feel your pain|",
		"Same, well obviosuly I don't know how you feel 100% but I thinks its appropiate to say I can relate to you deeply.|",
		"Hellow, is bad to|you know, even tho existence itself has no meaning, we can just try and take the best out of it as long as we can.|",
		"same man I feel you|",
		"If you want to talk about it more or anything, feel free to ask anything.|",
		"I totally feel you, you are not alone|",
		"feel free to message me if you want.|",
		"You deserve it! No one does!|",
		"Talk it all out here if you need to|PM if you need help or need to talk, always willing to!|YOU ARE NOT ALONE!|",
		"Dude don't say that man your family loves you|",
		"I'm here, and I haven't forgotten about you. I'm here to help!|",
		"sorry your post is just so relatable|",
		"I know what you're feeling|",
		"There are people who care about you.|",
		"I hope you are able to pass through this time and feel better.|",
		"I feel you|",
		"If you ever need to talk, feel free to message me.|",
		"Oh crap dude me too|",
		"If you want you can talk to me. Rant it out, listening|",
		"so sorry for what pain you are in|it just makes me mad that anyone has to feel this way about themselves.|",
		"not alone. PM me if you ever need someone to chat.|",
		"I'm sorry it didn't work out|",
		"Sorry to hear that|",
		"Stay strong friend Love|",
		"I feel for you|",
		"I promise you out there there is someone who wants to love you.|There are always people who want to be your friend|I hope that you wanna talk more|",
		"I'm sorry, that is so sad!|",
		"I know the feeling.|",
		"You are not alone. Just remember that.|",
		"I agree|",
		"I can relate|",
		"Hope you feel better|",
		"the road gets a little brighter. May you walk on warm sands, traveler.|",
		"It could be a medical condition, and you shouldn't be ashamed.|",
		"I support and am thankful for you, friend.|",
		"Message me at any time, we are all in this fight together.|",
		"if you do feel like talking about it more feel free to send me a message. Keep going with the journey and stay strong|",
		"I'm sorry.|",
		"I'm sorry man|",
		"But tunnels always have an end.|",
		"It's nice how you care for them|",
		"I'm sorry there isn't anything I can say to help|",
		"It will work out, things will get better but you have to try and make them better.|The truth is you're a loving caring compassionate kind person and deserves someone who wants you always|",
		"hey, wanna talk? im here for ya if ya want!|",
		"HAPPY BIRTHDAY!!!! I'm glad it's a good day!|"
	]
}

In [22]:
data['id'] = list(range(len(data['response_post'])))

In [23]:
input_dataset = Dataset.from_dict(data)

In [24]:
input_dataset

Dataset({
    features: ['response_post', 'rationales', 'id'],
    num_rows: 65
})

In [25]:
parser = StrOutputParser()

with open('configs/filepath_examples_rat.json', encoding='utf-8') as f:
	examples_data = json.load(f)

system_template = SYSTEM_MESSAGE
examples = examples_data["examples"]
system_template += "\n".join(
	f"Example Input: {json.dumps(example['input'], ensure_ascii=False, indent=2).replace('{', '{{').replace('}', '}}')}\n"
	f"Example Result: {json.dumps(example['result'], ensure_ascii=False, indent=2).replace('{', '{{').replace('}', '}}')}"
	for example in examples
)

prompt_template = ChatPromptTemplate.from_messages(
	[("system", system_template), ("user", "{text}")]
)

chain = prompt_template | model | parser

In [27]:
batched_input_dataset = split_into_batches(input_dataset)

In [28]:
list_of_results = []

In [ ]:
translate(batched_input_dataset)

In [30]:
if intermediate_path:
	d = Dataset.from_list(list_of_results)
	with open(intermediate_path, "w", encoding="utf-8") as f:
		json.dump(d.to_list(), f, ensure_ascii=False, indent=4)

In [31]:
with open('data/int_path_gpt-4o.json', 'r') as f:
    dd = json.load(f)

In [32]:
for item in dd:
    rats = item['rationales_rus'].split('|')
    for r in rats:
        if r not in item['response_post_rus']:
            print(item)